In [4]:
import numpy as np
import os
import cv2

In [22]:
train_subjects = np.loadtxt("../train_subjects_mask.txt")
len(train_subjects), train_subjects[:10]

(910, array([ 1.,  2.,  4.,  5.,  6.,  7.,  8.,  9., 11., 13.]))

In [24]:
n_classes = 1310
classes = np.arange(1, n_classes+1)

test_subjects = [c for c in classes if c not in train_subjects]
len(test_subjects)

400

In [25]:
from PIL import Image
data_path = "../UERC"

In [26]:
ear_data = os.listdir(data_path)

ear_imgs = {}
for person in ear_data:
    if int(person) not in test_subjects:
        continue
    
    imgs = os.listdir("%s/%s" % (data_path, person))
    try:
        ear_imgs[person] = [
            cv2.cvtColor(
                np.asarray(Image.open(f"{data_path}/{person}/{img}")), cv2.COLOR_BGR2RGB
            )
            for img in imgs
        ]
    except Exception as e:
        print(e)

In [29]:
X, y = [], []

for k, v in ear_imgs.items():
    y.extend(len(v) * [int(k)])
    X.extend(v)

In [34]:
len(X), len(y)

(75576, 75576)

In [35]:
from preprocess import resize_input

X = resize_input(X, 64, mode="test")

c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(
c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transfor

In [36]:
from ear_dataset import EarDataset

test_dataset = EarDataset(X, y)

In [37]:
import torch

In [38]:
torch.save(test_dataset, "data/test_dataset.pt")